In [46]:
import torch
import torchvision
import torchvision.transforms as transforms
import idx2numpy
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import PIL
import os
import random
import optuna
import joblib

In [47]:
trainimages = []
trainlabels = []
index = 0
mode = "Train"
Map= {}
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    Map[index]=emotion

    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            filepath =f'emotionfolder/{mode}/{emotion}/{file}'
            trainimages.append(filepath)
            trainlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [48]:
testimages = []
testlabels = []
index = 0
mode = 'Test'
emotions = os.listdir(f'emotionfolder/{mode}')
for emotion in emotions:
    if emotion=="affectnet" or emotion=="emotionfolder":
        continue
    
    files =os.listdir(f'emotionfolder/{mode}/{emotion}')
    for file in files:
        try:
            img = f'emotionfolder/{mode}/{emotion}/{file}'
            testimages.append(img)
            testlabels.append(index)
        except Exception as e:
            pass
    index+=1
    

In [49]:
#split is rougly 60/40 right now, need to make it ~80/10/10
trainimages.extend(testimages)
trainlabels.extend(testlabels)

trainimages_final = []
trainlabel_final = []
testimages_final = []
testlabel_final = []
valimages_final = []
vallabels_final = []

In [50]:

for i in range(0, len(trainimages)-1):
    num = random.randint(1,10)

    if 1<=num<=8:
        trainimages_final.append(trainimages[i])
        trainlabel_final.append(trainlabels[i])

    elif num==9:
        testimages_final.append(trainimages[i])
        testlabel_final.append(trainlabels[i])

    else:
        valimages_final.append(trainimages[i])
        vallabels_final.append(trainlabels[i])
        
    

In [114]:
target_size = (96,96)
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

In [115]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(target_size),
    transforms.RandomVerticalFlip(10),
    transforms.RandomHorizontalFlip(9),
    transforms.RandomRotation(12),
    transforms.ColorJitter(brightness=0.3, contrast=0.24),  
    transforms.ToTensor(),
    transforms.Normalize(mean, std)

])
transforms_test = transforms.Compose([
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

In [116]:
class MyDataset(Dataset):
    def __init__(self, imageslink, label,transformation_maker):
        super().__init__()
        self.imageslink = imageslink
        self.label = label
        self.transformation_maker = transformation_maker

    def __len__(self):
        return len(self.label)

    def __getitem__(self,idx):
        img = Image.open(self.imageslink[idx])
        img =  self.transformation_maker(img)

        return img, self.label[idx]

In [117]:
trainDataset = MyDataset(trainimages_final, trainlabel_final, transform_train)
testDataset = MyDataset(testimages_final, testlabel_final, transforms_test)
valDataset = MyDataset(valimages_final, vallabels_final, transforms_test)
trainloader = DataLoader(trainDataset, batch_size=256, shuffle=True, num_workers=16, pin_memory=True)
testloader = DataLoader(testDataset, batch_size=256, shuffle=True, num_workers=16, pin_memory=True)
valloader = DataLoader(valDataset, batch_size=256, shuffle=True, num_workers=16, pin_memory=True)
#8outputs

In [134]:
class CnnArchitecture(nn.Module):
    def __init__(self, is_grayscale, output_size, num_layers, hidden_size,  kernel_size, image_size=96):
        super().__init__()

        self.input_size=1 if is_grayscale else 3
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.imagesize = image_size
        self.kernelsize = kernel_size
        self.map = {3:1, 5:2, 7:3}
        self.Network = []

        for i in range(num_layers):
            if i==0:
                self.Network.append(nn.Conv2d(self.input_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

            else:
                self.Network.append(nn.Conv2d(self.hidden_size, self.hidden_size, kernel_size=self.kernelsize, padding=self.map[self.kernelsize]))
                self.Network.append(nn.ReLU())
                self.Network.append(nn.MaxPool2d(kernel_size=2, stride=2))

        self.features = nn.Sequential(*self.Network)
        sampletorch = torch.randn(1, self.input_size, self.imagesize, self.imagesize)
        self.dummy_size = self.features(sampletorch).numel()

        self.classifier = nn.Linear(self.dummy_size, self.output_size)

    def forward(self,x):
        x=self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)#no relu here since criterion and loss will take care of it!
                                                                              

                                                                              
                                                            
                                                
        

In [119]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [120]:
model = CnnArchitecture(False, 8, 3, 32)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [121]:
epochs = 200
for i in range(epochs):
    model.train()
    training_loss = 0
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        training_loss+=loss.item()
        loss.backward()
        optimizer.step()


    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in valloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss+=loss.item()
            total += y.size(0)
            probability, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == y).sum().item()
            
    if True: 
        avg_train_loss = training_loss / len(trainloader)
        avg_test_loss = test_loss / len(valloader)
        accuracy = (100 * correct) / total

        print(f"Epoch num: {i}")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Test Loss:  {avg_test_loss:.4f}")
        print(f"  Accuracy:   {accuracy:.2f}%")
        print("--------------------")            

Epoch num: 0
  Train Loss: 2.0206
  Test Loss:  2.0373
  Accuracy:   19.15%
--------------------
Epoch num: 1
  Train Loss: 1.9652
  Test Loss:  2.0550
  Accuracy:   18.55%
--------------------
Epoch num: 2
  Train Loss: 1.9225
  Test Loss:  2.0008
  Accuracy:   19.28%
--------------------
Epoch num: 3
  Train Loss: 1.8754
  Test Loss:  2.0658
  Accuracy:   19.84%
--------------------
Epoch num: 4
  Train Loss: 1.7964
  Test Loss:  1.9075
  Accuracy:   23.61%
--------------------
Epoch num: 5
  Train Loss: 1.7209
  Test Loss:  1.8237
  Accuracy:   27.78%
--------------------
Epoch num: 6
  Train Loss: 1.6677
  Test Loss:  1.8666
  Accuracy:   27.15%
--------------------
Epoch num: 7
  Train Loss: 1.6317
  Test Loss:  1.8217
  Accuracy:   27.58%
--------------------
Epoch num: 8
  Train Loss: 1.6098
  Test Loss:  1.8448
  Accuracy:   26.65%
--------------------
Epoch num: 9
  Train Loss: 1.5898
  Test Loss:  1.7992
  Accuracy:   28.11%
--------------------
Epoch num: 10
  Train Loss: 1.

Exception ignored in: <function _afterFork at 0x7fe56a3b3740>
Traceback (most recent call last):
  File "/usr/lib/python3.13/logging/__init__.py", line 245, in _afterFork
    def _afterFork():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 239438, 239439, 239440, 239441, 239442, 239443, 239444, 239445) exited unexpectedly

In [139]:
2**6

64

In [140]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32,64,128,256])
    num_layers = trial.suggest_categorical("num_layers", [2,4,6])
    hidden_size = trial.suggest_categorical("hidden_size", [16, 64, 128])
    lr = trial.suggest_categorical("lr", [1e-3, 1e-4, 1e-5])
    epochs = trial.suggest_categorical("epochs", [50, 75, 100])
    weight_decay = trial.suggest_categorical("weight_decay", [1e-2, 1e-3, 1e-4])   
    kernel_size = trial.suggest_categorical("kernel_size", [3,5,7])

    

    
    trainloader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    valloader = DataLoader(valDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    testloss = []

    
    for i in range(epochs):
        model.train()
        training_loss = 0
        for x,y in trainloader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            training_loss+=loss.item()
            loss.backward()
            optimizer.step()
    
    
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for x,y in valloader:
                x = x.to(device)
                y = y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                test_loss+=loss.item()
                total += y.size(0)
                probability, predicted = torch.max(y_pred.data, 1)
                correct += (predicted == y).sum().item()
                
        if i%20==0 or i==epochs-1: 
            avg_train_loss = training_loss / len(trainloader)
            avg_test_loss = test_loss / len(valloader)
            accuracy = (100 * correct) / total
    
            print(f"Epoch num: {i}")
            print(f"  Train Loss: {avg_train_loss}")
            print(f"  Test Loss:  {avg_test_loss}")
            print(f"  Accuracy:   {accuracy}%")
            print("--------------------")
            testloss.append(avg_test_loss)
            trial.report(avg_test_loss, i)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return testloss[-1]#want to minimize testloss!


In [142]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=2)

[I 2025-11-10 21:02:16,733] A new study created in memory with name: no-name-1607bd90-eb2a-4e16-948d-619da8e3c268


Epoch num: 0
  Train Loss: 2.00931101088113
  Test Loss:  2.066826912264029
  Accuracy:   17.658730158730158%
--------------------
Epoch num: 20
  Train Loss: 1.5989997386932373
  Test Loss:  1.9629530683159828
  Accuracy:   25.363756613756614%
--------------------
Epoch num: 40
  Train Loss: 1.469299106311549
  Test Loss:  2.2151202087601027
  Accuracy:   22.056878306878307%
--------------------
Epoch num: 60
  Train Loss: 1.4036296089386504
  Test Loss:  2.118182343741258
  Accuracy:   22.71825396825397%
--------------------


[I 2025-11-10 21:15:35,661] Trial 0 finished with value: 2.1473116725683212 and parameters: {'batch_size': 64, 'num_layers': 2, 'hidden_size': 64, 'lr': 0.0001, 'epochs': 75, 'weight_decay': 0.001, 'kernel_size': 5}. Best is trial 0 with value: 2.1473116725683212.


Epoch num: 74
  Train Loss: 1.3735756814946705
  Test Loss:  2.1473116725683212
  Accuracy:   22.321428571428573%
--------------------
Epoch num: 0
  Train Loss: 2.0273649605385007
  Test Loss:  2.0424024015665054
  Accuracy:   18.154761904761905%
--------------------
Epoch num: 20
  Train Loss: 1.3194255900445246
  Test Loss:  1.8917801255981128
  Accuracy:   26.256613756613756%
--------------------
Epoch num: 40
  Train Loss: 1.212023225372205
  Test Loss:  1.8358566612005234
  Accuracy:   31.547619047619047%
--------------------


[I 2025-11-10 21:25:41,432] Trial 1 finished with value: 1.744110345840454 and parameters: {'batch_size': 64, 'num_layers': 6, 'hidden_size': 64, 'lr': 0.001, 'epochs': 50, 'weight_decay': 0.0001, 'kernel_size': 5}. Best is trial 1 with value: 1.744110345840454.


Epoch num: 49
  Train Loss: 1.1887231570311065
  Test Loss:  1.744110345840454
  Accuracy:   30.91931216931217%
--------------------


In [143]:
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

    batch_size: 64
    num_layers: 6
    hidden_size: 64
    lr: 0.001
    epochs: 50
    weight_decay: 0.0001
    kernel_size: 5


In [145]:
#now build model with best found params
best_params = study.best_params
batch_size = best_params["batch_size"]
num_layers = best_params["num_layers"]
hidden_size = best_params["hidden_size"]
lr = best_params["lr"]
epochs = best_params["epochs"]
kernel_size = best_params["kernel_size"]
weight_decay = best_params["weight_decay"]

In [148]:
#custom dataset with both train && val
compressedtrain = []
compressedtrainlabels = []
compressedtrain.extend(trainimages_final)
compressedtrain.extend(valimages_final)
compressedtrainlabels.extend(trainlabel_final)
compressedtrainlabels.extend(vallabels_final)
compressedDataset =  MyDataset(compressedtrain, compressedtrainlabels, transform_train)



trainloader = DataLoader(compressedDataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CnnArchitecture(False, 8, num_layers, hidden_size, kernel_size)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
testloss = []


for i in range(1):
    model.train()
    training_loss = 0
    for x,y in trainloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        training_loss+=loss.item()
        loss.backward()
        optimizer.step()


    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in valloader:
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            test_loss+=loss.item()
            total += y.size(0)
            probability, predicted = torch.max(y_pred.data, 1)
            correct += (predicted == y).sum().item()
            
    if i%20==0 or i==epochs-1: 
        avg_train_loss = training_loss / len(trainloader)
        avg_test_loss = test_loss / len(valloader)
        accuracy = (100 * correct) / total

        print(f"Epoch num: {i}")
        print(f"  Train Loss: {avg_train_loss}")
        print(f"  Test Loss:  {avg_test_loss}")
        print(f"  Accuracy:   {accuracy}%")
        print("--------------------")
        testloss.append(avg_test_loss)

Epoch num: 0
  Train Loss: 2.0322392606403326
  Test Loss:  2.06941290696462
  Accuracy:   15.211640211640212%
--------------------


In [149]:
joblib.dump(model, "cnn_emotion_prediction.pkl")

['cnn_emotion_prediction.pkl']

In [173]:
device = torch.device('cuda')
model = joblib.load('cnn_emotion_prediction.pkl')
model = model.to(device)

In [174]:
#testrun for prediction
img = Image.open('emotionfolder/Train/anger/image0022969.jpg')
img = transforms_test(img)
img = img.unsqueeze(0)
img = img.to(device)
model.eval()
with torch.no_grad():
    y_pred = model(img)

In [188]:

EmotionMap = {v:k for k,v in Map.items()}

In [189]:
EmotionMap

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happy': 4,
 'neutral': 5,
 'sad': 6,
 'surprise': 7}

In [192]:
#need to define this into helper file later on...
def Predict(imgpath, model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    PredictionEmotionMap =EmotionMap.copy()
    img = Image.open(imgpath)
    img = transforms_test(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        y_pred = model(img)

    prediction_confidence_array = torch.nn.functional.softmax(y_pred, dim=1)[0].cpu().numpy()
    for i in range(len(prediction_confidence_array)):
        emotion = Map[i]
        PredictionEmotionMap[emotion] = prediction_confidence_array[i]

    return PredictionEmotionMap
    #how would I convert confidence to a 100%scale cause it ain't like that right now no?
    

In [193]:
Predict('emotionfolder/Train/anger/image0022969.jpg', model)

{'anger': np.float32(0.10275879),
 'contempt': np.float32(0.10070733),
 'disgust': np.float32(0.09365553),
 'fear': np.float32(0.10301596),
 'happy': np.float32(0.16819917),
 'neutral': np.float32(0.15946205),
 'sad': np.float32(0.13645828),
 'surprise': np.float32(0.13574292)}